In [38]:
import pandas as pd
import os
import plotly.express as px

from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Connexion NeonDB

In [39]:
load_dotenv("secrets.env")

True

In [40]:
NEON_USER=os.getenv("NEON_USER")
NEON_PASSWORD=os.getenv("NEON_PASSWORD")
NEON_HOST=os.getenv("NEON_HOST")
NEON_DBNAME=os.getenv("NEON_DBNAME")
print("🔍 Vérification des variables :")
print(f"NEON_USER={NEON_USER}")
print(f"NEON_PASSWORD={'*' * len(NEON_PASSWORD) if NEON_PASSWORD else None}")
print(f"NEON_HOST={NEON_HOST}")
print(f"NEON_DBNAME={NEON_DBNAME}")

engine = create_engine(
    f"postgresql+psycopg2://{NEON_USER}:{NEON_PASSWORD}@{NEON_HOST}/{NEON_DBNAME}?sslmode=require"
)

🔍 Vérification des variables :
NEON_USER=neondb_owner
NEON_PASSWORD=****************
NEON_HOST=ep-billowing-mode-ag8c4ezd-pooler.c-2.eu-central-1.aws.neon.tech
NEON_DBNAME=neondb


## Requête essai

In [41]:
with engine.connect() as conn:
    df_villes = pd.read_sql("SELECT * FROM villes", engine)
    df_meteos = pd.read_sql("SELECT * from meteos", engine)
    df_hotels = pd.read_sql("SELECT * from hotels", engine)
    df_condition = pd.read_sql("SELECT * FROM condition", engine)
    

In [42]:
display(df_villes.head(5))
display(df_hotels.head(5))
display(df_meteos.head(5))
display(df_condition.head(5))


,ville_id,nom,latitude,longitude,latitude2,longitude2
0,1,Le Mont Saint Michel,48.635523,-1.510257,48.635523,-1.510257
1,2,Saint Malo,48.649518,-2.026041,48.649518,-2.026041
2,3,Bayeux,49.276462,-0.702474,49.276462,-0.702474
3,4,Le Havre,49.493898,0.107973,49.493898,0.107973
4,5,Rouen,49.440459,1.093966,49.440459,1.093966


,ville,name,description,url,rating,latitude,longitude,prix,ville_id
0,Aigues-Mortes,Hotel De La Pyramide,"Situé sur les rives de l'étang de Thau, l'Hôte...",https://www.booking.com/hotel/fr/de-la-pyramid...,"Avec une note de 9,2",43.417488,3.595718,€ 724,26
1,Aigues-Mortes,Abrivado,"Situé dans le parc régional de Camargue, l'Abr...",https://www.booking.com/hotel/fr/abrivado.fr.h...,"Avec une note de 8,8",43.450337,4.426224,€ 784,26
2,Aigues-Mortes,La Maison de Lyna - Suites de charme intra-mur...,L’établissement La Maison de Lyna - Suites de ...,https://www.booking.com/hotel/fr/la-maison-de-...,"Avec une note de 9,0",43.566018,4.192353,€ 1 019,26
3,Aigues-Mortes,Hôtel Horizon Resort Massane & Spa,L’établissement Hôtel Horizon Resort Massane &...,https://www.booking.com/hotel/fr/horizon-resor...,"Avec une note de 9,1",43.649344,4.001001,€ 1 011,26
4,Aigues-Mortes,La Villa Mazarin,La Villa Mazarin vous accueille dans un bâtime...,https://www.booking.com/hotel/fr/la-villa-maza...,"Avec une note de 9,2",43.564987,4.191752,€ 929,26


,ville_id,date,condition,temperature,duree_ensoleillement,pluie,vent
0,1,2025-11-15,6,12.147470,3137.1309,0.7,26.178707
1,1,2025-11-16,1,8.873925,11569.6260,0.2,28.673721
2,1,2025-11-17,1,6.984726,28183.8710,0.0,7.952660
3,1,2025-11-18,3,5.724897,22818.7170,0.7,33.563120
4,1,2025-11-19,3,3.097008,28047.7340,0.5,34.979332


,description,code
0,Soleil,1
1,Nuageux,2
2,Bruine,3
3,Pluie,4
4,Neige,5


# Top 5 des villes pour la semaine du 14 au 21 Novembre

In [43]:
query = """
SELECT 
    v.nom,
    v.latitude,
    v.longitude,
    m.temperature,
    m.duree_soleil,
    m.vent,
    m.pluie,
	c.code,
    c.description temps,
    v.ville_id
FROM villes v
	INNER JOIN 
		(
			SELECT 
				ville_id, 
				avg(temperature) temperature, 
				avg(duree_ensoleillement) duree_soleil, 
				avg(vent) vent,
				avg(pluie)pluie, 
				round(avg(condition),0) condition
			FROM meteos
			GROUP BY
				ville_id
		) m on v.ville_id=m.ville_id
	INNER JOIN condition c ON m.condition=c.code
ORDER BY 
    c.code,
	m.duree_soleil DESC,
	m.temperature DESC,
	m.pluie,
	m.vent
;
"""

with engine.connect() as conn:
    result = conn.execute(text(query))
    top5_destinations_sql = pd.DataFrame(result.fetchall(), columns=result.keys())

top5 = top5_destinations_sql.head(5).copy()
top5["rang"] = range(1, len(top5) + 1)
top5

,nom,latitude,longitude,temperature,duree_soleil,vent,pluie,code,temps,ville_id,rang
0,Collioure,42.525050,3.083155,12.693421,29298.445250,31.058938,0.975000,2,Nuageux,28,1
1,Cassis,43.214036,5.539632,10.267872,26778.400125,37.981552,4.737501,2,Nuageux,20,2
2,La palud sur Verdon,43.779686,6.341385,5.122990,26000.339625,12.661177,4.725000,2,Nuageux,18,3
3,Aigues-Mortes,43.566152,4.191540,10.722876,24950.529625,25.509239,2.200000,2,Nuageux,26,4
4,Saintes-Maries-de-la-mer,43.451592,4.427720,10.719892,24915.940875,25.591512,2.662500,2,Nuageux,27,5


In [44]:
import plotly.express as px

top5["rang_str"] = top5["rang"].astype(str)

custom_colors = {
    "1": "blue",
    "2": "red",
    "3": "green",
    "4": "orange",
    "5": "purple"
}


fig = px.scatter_mapbox(
    top5,
    lat="latitude",
    lon="longitude",
    size="temperature",
    color="rang_str",  
    color_discrete_map=custom_colors,
    hover_name="nom",
    hover_data={
        "temperature": True,
        "duree_soleil": True,
        "vent": True,
        "pluie": True,
        "temps": True,
        "rang": False,
        "latitude": False,
        "longitude": False
    },
    mapbox_style="open-street-map",
    zoom=4,
    title="Classement des destinations selon la météo"
)

fig.update_layout(
    legend_title_text="Classement météo",
    width=550,   # largeur en pixels (par défaut c’est ~1100)
    height=600,  # optionnel, tu peux aussi réduire la hauteur
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.25,
        xanchor="center",
        x=0.5,
        font=dict(size=13, color="black"),
        bgcolor="rgba(255,255,255,0.8)",
        bordercolor="black",
        borderwidth=1
    )
)

fig.show()


## 20 hotels et où ils sont

In [45]:
top5

,nom,latitude,longitude,temperature,duree_soleil,vent,pluie,code,temps,ville_id,rang,rang_str
0,Collioure,42.525050,3.083155,12.693421,29298.445250,31.058938,0.975000,2,Nuageux,28,1,1
1,Cassis,43.214036,5.539632,10.267872,26778.400125,37.981552,4.737501,2,Nuageux,20,2,2
2,La palud sur Verdon,43.779686,6.341385,5.122990,26000.339625,12.661177,4.725000,2,Nuageux,18,3,3
3,Aigues-Mortes,43.566152,4.191540,10.722876,24950.529625,25.509239,2.200000,2,Nuageux,26,4,4
4,Saintes-Maries-de-la-mer,43.451592,4.427720,10.719892,24915.940875,25.591512,2.662500,2,Nuageux,27,5,5


In [46]:
top5_villes = top5['ville_id'].tolist()

for ville in top5_villes:
    query = f"""
        SELECT 
            h.name AS hotel,
            v.nom,
            h.latitude,
            h.longitude,
            CAST(REPLACE(SUBSTR(h.rating, 18), ',', '.') AS FLOAT) AS note,
            prix
        FROM villes v
        INNER JOIN hotels h ON h.ville_id = v.ville_id
        WHERE v.ville_id = {ville}
        ORDER BY note DESC
    """

    with engine.connect() as conn:
        result = conn.execute(text(query))
        top_hotels = pd.DataFrame(result.fetchall(), columns=result.keys())

    if top_hotels.empty:
        print(f"⚠️ Pas de données pour ville_id={ville}")
        continue

    top_hotels = top_hotels.dropna(subset=["note"])

    # ✅ Création de la carte
    fig = px.scatter_map(
        top_hotels,
        lat="latitude",
        width=500,
        height=500,
        lon="longitude",
        hover_name="hotel",
        hover_data={"nom": True, "note": True, "prix":True, "latitude": False, "longitude": False},
        color="note",
        color_continuous_scale="Viridis",
        range_color=[7, 10], 
        size="note",         # les points dépendent de la note
        size_max=20,         # 👈 augmente la taille maximale
        zoom=9,             # 👈 zoom plus serré sur la ville
        title=f"🏨 Hôtels à {top_hotels['nom'].iloc[0]}"
    )

    fig.update_layout(
        coloraxis_colorbar=dict(title="Note"),
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()
